##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Rédaction de vos propres rappels

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/keras/custom_callback"><img src="https://www.tensorflow.org/images/tf_logo_32px.png"> Voir sur TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/keras-team/keras-io/blob/master/tf/custom_callback.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png"> Exécuter dans Google Colab</a></td>
  <td><a target="_blank" href="https://github.com/keras-team/keras-io/blob/master/guides/writing_your_own_callbacks.py"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png"> Afficher la source sur GitHub</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/keras-io/tf/custom_callback.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png"> Télécharger le cahier</a></td>
</table>

## introduction

Un rappel est un outil puissant pour personnaliser le comportement d'un modèle Keras pendant l'entraînement, l'évaluation ou l'inférence. Les exemples incluent `tf.keras.callbacks.TensorBoard` pour visualiser la progression et les résultats de l'entraînement avec TensorBoard, ou `tf.keras.callbacks.ModelCheckpoint` pour enregistrer périodiquement votre modèle pendant l'entraînement.

Dans ce guide, vous apprendrez ce qu'est un rappel Keras, ce qu'il peut faire et comment vous pouvez créer le vôtre. Nous proposons quelques démos d'applications de rappel simples pour vous aider à démarrer.

## Installer

In [ ]:
import tensorflow as tf
from tensorflow import keras

## Présentation des rappels Keras

All callbacks subclass the `keras.callbacks.Callback` class, and override a set of methods called at various stages of training, testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training.

Vous pouvez transmettre une liste de rappels (en tant que `callbacks` argument de mot-clé) aux méthodes de modèle suivantes:

- `keras.Model.fit()`
- `keras.Model.evaluate()`
- `keras.Model.predict()`

## Un aperçu des méthodes de rappel

### Méthodes globales

#### `on_(train|test|predict)_begin(self, logs=None)`

Appelé au début de l' `fit` / `evaluate` / `predict` .

#### `on_(train|test|predict)_end(self, logs=None)`

Appelé à la fin de l' `fit` / `evaluate` / `predict` .

### Méthodes au niveau du lot pour la formation / le test / la prévision

#### `on_(train|test|predict)_batch_begin(self, batch, logs=None)`

Appelé juste avant de traiter un lot pendant la formation / les tests / les prévisions.

#### `on_(train|test|predict)_batch_end(self, batch, logs=None)`

Appelé à la fin de la formation / test / prédiction d'un lot. Dans cette méthode, `logs` est un dict contenant les résultats des métriques.

### Méthodes au niveau de l'époque (formation uniquement)

#### `on_epoch_begin(self, epoch, logs=None)`

Appelé au début d'une époque pendant l'entraînement.

#### `on_epoch_end(self, epoch, logs=None)`

Appelé à la fin d'une époque pendant l'entraînement.

## Un exemple basique

Prenons un exemple concret. Pour commencer, importons tensorflow et définissons un modèle Keras séquentiel simple:

In [ ]:
# Define the Keras model to add callbacks to
def get_model():
    model = keras.Sequential()
    model.add(keras.layers.Dense(1, input_dim=784))
    model.compile(
        optimizer=keras.optimizers.RMSprop(learning_rate=0.1),
        loss="mean_squared_error",
        metrics=["mean_absolute_error"],
    )
    return model


Ensuite, chargez les données MNIST pour l'entraînement et les tests à partir de l'API des ensembles de données Keras:

In [ ]:
# Load example MNIST data and pre-process it
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(-1, 784).astype("float32") / 255.0
x_test = x_test.reshape(-1, 784).astype("float32") / 255.0

# Limit the data to 1000 samples
x_train = x_train[:1000]
y_train = y_train[:1000]
x_test = x_test[:1000]
y_test = y_test[:1000]

Maintenant, définissez un simple rappel personnalisé qui enregistre:

- Quand `fit` / `evaluate` / `predict` début et la fin
- Quand chaque époque commence et se termine
- Quand chaque lot d'entraînement commence et se termine
- Quand chaque lot d'évaluation (test) commence et se termine
- Quand chaque lot d'inférence (prédiction) commence et se termine

In [ ]:
class CustomCallback(keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        keys = list(logs.keys())
        print("Starting training; got log keys: {}".format(keys))

    def on_train_end(self, logs=None):
        keys = list(logs.keys())
        print("Stop training; got log keys: {}".format(keys))

    def on_epoch_begin(self, epoch, logs=None):
        keys = list(logs.keys())
        print("Start epoch {} of training; got log keys: {}".format(epoch, keys))

    def on_epoch_end(self, epoch, logs=None):
        keys = list(logs.keys())
        print("End epoch {} of training; got log keys: {}".format(epoch, keys))

    def on_test_begin(self, logs=None):
        keys = list(logs.keys())
        print("Start testing; got log keys: {}".format(keys))

    def on_test_end(self, logs=None):
        keys = list(logs.keys())
        print("Stop testing; got log keys: {}".format(keys))

    def on_predict_begin(self, logs=None):
        keys = list(logs.keys())
        print("Start predicting; got log keys: {}".format(keys))

    def on_predict_end(self, logs=None):
        keys = list(logs.keys())
        print("Stop predicting; got log keys: {}".format(keys))

    def on_train_batch_begin(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Training: start of batch {}; got log keys: {}".format(batch, keys))

    def on_train_batch_end(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Training: end of batch {}; got log keys: {}".format(batch, keys))

    def on_test_batch_begin(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Evaluating: start of batch {}; got log keys: {}".format(batch, keys))

    def on_test_batch_end(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Evaluating: end of batch {}; got log keys: {}".format(batch, keys))

    def on_predict_batch_begin(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Predicting: start of batch {}; got log keys: {}".format(batch, keys))

    def on_predict_batch_end(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Predicting: end of batch {}; got log keys: {}".format(batch, keys))


Essayons-le:

In [ ]:
model = get_model()
model.fit(
    x_train,
    y_train,
    batch_size=128,
    epochs=1,
    verbose=0,
    validation_split=0.5,
    callbacks=[CustomCallback()],
)

res = model.evaluate(
    x_test, y_test, batch_size=128, verbose=0, callbacks=[CustomCallback()]
)

res = model.predict(x_test, batch_size=128, callbacks=[CustomCallback()])

### Utilisation des `logs` dict

Le `logs` dict contient la valeur de la perte et toutes les métriques à la fin d'un lot ou d'une époque. L'exemple comprend la perte et l'erreur absolue moyenne.

In [ ]:
class LossAndErrorPrintingCallback(keras.callbacks.Callback):
    def on_train_batch_end(self, batch, logs=None):
        print("For batch {}, loss is {:7.2f}.".format(batch, logs["loss"]))

    def on_test_batch_end(self, batch, logs=None):
        print("For batch {}, loss is {:7.2f}.".format(batch, logs["loss"]))

    def on_epoch_end(self, epoch, logs=None):
        print(
            "The average loss for epoch {} is {:7.2f} "
            "and mean absolute error is {:7.2f}.".format(
                epoch, logs["loss"], logs["mean_absolute_error"]
            )
        )


model = get_model()
model.fit(
    x_train,
    y_train,
    batch_size=128,
    epochs=2,
    verbose=0,
    callbacks=[LossAndErrorPrintingCallback()],
)

res = model.evaluate(
    x_test,
    y_test,
    batch_size=128,
    verbose=0,
    callbacks=[LossAndErrorPrintingCallback()],
)

## Utilisation de l'attribut `self.model`

En plus de recevoir des informations de journal lorsque l'une de leurs méthodes est appelée, les callbacks ont accès au modèle associé à la ronde actuelle de formation / évaluation / inférence: `self.model` .

Voici quelques-unes des choses que vous pouvez faire avec `self.model` dans un rappel:

- Définissez `self.model.stop_training = True` pour interrompre immédiatement l'entraînement.
- Mutation des hyperparamètres de l'optimiseur (disponibles sous le nom de `self.model.optimizer` ), tels que `self.model.optimizer.learning_rate` .
- Enregistrez le modèle à intervalles réguliers.
- Enregistrez la sortie de `model.predict()` sur quelques échantillons de test à la fin de chaque époque, à utiliser comme contrôle de cohérence pendant la formation.
- Extrayez des visualisations des caractéristiques intermédiaires à la fin de chaque époque, pour surveiller ce que le modèle apprend au fil du temps.
- etc.

Voyons cela en action dans quelques exemples.

## Exemples d'applications de rappel Keras

### Arrêt précoce avec une perte minimale

Ce premier exemple montre la création d'un `Callback` qui arrête l'entraînement lorsque le minimum de perte a été atteint, en définissant l'attribut `self.model.stop_training` (booléen). En option, vous pouvez fournir un argument `patience` pour spécifier combien d'époques nous devons attendre avant de s'arrêter après avoir atteint un minimum local.

`tf.keras.callbacks.EarlyStopping` fournit une implémentation plus complète et plus générale.

In [ ]:
import numpy as np


class EarlyStoppingAtMinLoss(keras.callbacks.Callback):
    """Stop training when the loss is at its min, i.e. the loss stops decreasing.

  Arguments:
      patience: Number of epochs to wait after min has been hit. After this
      number of no improvement, training stops.
  """

    def __init__(self, patience=0):
        super(EarlyStoppingAtMinLoss, self).__init__()
        self.patience = patience
        # best_weights to store the weights at which the minimum loss occurs.
        self.best_weights = None

    def on_train_begin(self, logs=None):
        # The number of epoch it has waited when loss is no longer minimum.
        self.wait = 0
        # The epoch the training stops at.
        self.stopped_epoch = 0
        # Initialize the best as infinity.
        self.best = np.Inf

    def on_epoch_end(self, epoch, logs=None):
        current = logs.get("loss")
        if np.less(current, self.best):
            self.best = current
            self.wait = 0
            # Record the best weights if current results is better (less).
            self.best_weights = self.model.get_weights()
        else:
            self.wait += 1
            if self.wait >= self.patience:
                self.stopped_epoch = epoch
                self.model.stop_training = True
                print("Restoring model weights from the end of the best epoch.")
                self.model.set_weights(self.best_weights)

    def on_train_end(self, logs=None):
        if self.stopped_epoch > 0:
            print("Epoch %05d: early stopping" % (self.stopped_epoch + 1))


model = get_model()
model.fit(
    x_train,
    y_train,
    batch_size=64,
    steps_per_epoch=5,
    epochs=30,
    verbose=0,
    callbacks=[LossAndErrorPrintingCallback(), EarlyStoppingAtMinLoss()],
)

### Planification du taux d'apprentissage

Dans cet exemple, nous montrons comment un rappel personnalisé peut être utilisé pour modifier dynamiquement le taux d'apprentissage de l'optimiseur au cours de la formation.

Voir `callbacks.LearningRateScheduler` pour une implémentation plus générale.

In [ ]:
class CustomLearningRateScheduler(keras.callbacks.Callback):
    """Learning rate scheduler which sets the learning rate according to schedule.

  Arguments:
      schedule: a function that takes an epoch index
          (integer, indexed from 0) and current learning rate
          as inputs and returns a new learning rate as output (float).
  """

    def __init__(self, schedule):
        super(CustomLearningRateScheduler, self).__init__()
        self.schedule = schedule

    def on_epoch_begin(self, epoch, logs=None):
        if not hasattr(self.model.optimizer, "lr"):
            raise ValueError('Optimizer must have a "lr" attribute.')
        # Get the current learning rate from model's optimizer.
        lr = float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))
        # Call schedule function to get the scheduled learning rate.
        scheduled_lr = self.schedule(epoch, lr)
        # Set the value back to the optimizer before this epoch starts
        tf.keras.backend.set_value(self.model.optimizer.lr, scheduled_lr)
        print("\nEpoch %05d: Learning rate is %6.4f." % (epoch, scheduled_lr))


LR_SCHEDULE = [
    # (epoch to start, learning rate) tuples
    (3, 0.05),
    (6, 0.01),
    (9, 0.005),
    (12, 0.001),
]


def lr_schedule(epoch, lr):
    """Helper function to retrieve the scheduled learning rate based on epoch."""
    if epoch < LR_SCHEDULE[0][0] or epoch > LR_SCHEDULE[-1][0]:
        return lr
    for i in range(len(LR_SCHEDULE)):
        if epoch == LR_SCHEDULE[i][0]:
            return LR_SCHEDULE[i][1]
    return lr


model = get_model()
model.fit(
    x_train,
    y_train,
    batch_size=64,
    steps_per_epoch=5,
    epochs=15,
    verbose=0,
    callbacks=[
        LossAndErrorPrintingCallback(),
        CustomLearningRateScheduler(lr_schedule),
    ],
)

### Callbacks Keras intégrés

Be sure to check out the existing Keras callbacks by reading the [API docs](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/). Applications include logging to CSV, saving the model, visualizing metrics in TensorBoard, and a lot more!